In [20]:
import pandas as pd
import numpy as np
import time as t
from pathlib import Path
from xlwings import view

In [21]:
time_stamp = t.time()

In [25]:
# Source files
file_path_src_pq            = Path("D:/Analytik/3_ФТТ/", "_src_pq.xlsx")
file_path_src_ftt           = Path("D:/Analytik/3_ФТТ/_исх/2024/5/", "фтт.xlsx")
file_path_src_ttzz_squ      = Path("D:/Analytik/3_ФТТ/_исх/2024/5/", "ттзз_скю.xlsx")

# Manuals dataframe
shimpments_qnt_df           = pd.read_excel(file_path_src_pq, "Количество отгрузок")
departments_df              = pd.read_excel(file_path_src_pq, "Справочник отделов")

# Source dataframe
src_ftt_df                  = pd.read_excel(file_path_src_ftt, header=[0])
src_ttzz_squ                = pd.read_excel(file_path_src_ttzz_squ, header=[0])
# Amounting month
month                       = t.strftime("01.05.2024")

In [26]:
src_ftt_df

,Подразделение,СВ_Команда (Пользователи),Код ТА (Пользователи),Менеджер,Клиент,Заказ клиента / Реализация.Дата отгрузки,Выручка+руч.скидка,"Вес (нетто), кг"
0,Беларусская Ферма ОП,NaN,NaN,Абдуллаев Аваз Муртуз Оглы,10287.Роллет № 86,01.05.2024,3512.38,12.441
1,Беларусская Ферма ОП,NaN,NaN,Абдуллаев Аваз Муртуз Оглы,10287.Роллет № 86,04.05.2024,1862.09,13.960
2,Беларусская Ферма ОП,NaN,NaN,Абдуллаев Аваз Муртуз Оглы,10287.Роллет № 86,15.05.2024,2142.51,6.900
3,Беларусская Ферма ОП,NaN,NaN,Абдуллаев Аваз Муртуз Оглы,10287.Роллет № 86,24.05.2024,2259.62,15.820
4,Беларусская Ферма ОП,NaN,NaN,Абдуллаев Аваз Муртуз Оглы,10311.маг. Родничок,08.05.2024,2568.43,19.788
...,...,...,...,...,...,...,...,...
21382,Главмолоко ОП,ГМ Область,118018.0,Котух Андрей Валериевич,5342.маг. Гранд 1,24.05.2024,5233.59,14.500
21383,Главмолоко ОП,ГМ Область,118018.0,Котух Андрей Валериевич,5342.маг. Гранд 1,31.05.2024,1617.04,16.000
21384,Главмолоко ОП,ГМ Область,118018.0,Котух Андрей Валериевич,8302.Светлячек-2,23.05.2024,2074.69,5.060
21385,Главмолоко ОП,ГМ Область,118022.0,Горная Анна Викторовна,30298.маг. Шахтерская птицефабрика № 52,30.05.2024,2365.08,21.930


In [28]:
# Количество отгрузок
shimpments_qnt_df = shimpments_qnt_df.loc[shimpments_qnt_df["Начало месяца"] == month]
shimpments_qnt_df.columns = ["Отдел", "Начало месяца", "Значение", "Атрибут"]

# Преобразование object в date
shimpments_qnt_df['Начало месяца'] = pd.to_datetime(shimpments_qnt_df['Начало месяца'], format='%d.%m.%Y')
shimpments_qnt_df = shimpments_qnt_df.set_index(["Отдел", "Начало месяца"])
shimpments_qnt_df

,,Значение,Атрибут
Отдел,Начало месяца,,
Главмолоко,2024-05-01,26,Количество отгрузок
ФЕРМА,2024-05-01,26,Количество отгрузок
VIP,2024-05-01,26,Количество отгрузок


In [62]:
df = src_ftt_df.dropna()

# Список полей для переименования
columns = ['Отдел продаж', 'Подразделение', 'Должность ТА', 'ФИО ТА', 'РТТ', 'Дата', 'Значение', 'Вес']
df.columns = columns

# Преобразование object в date
df['Дата'] = pd.to_datetime(df['Дата'], format='%d.%m.%Y')

df = pd.merge(df, departments_df, on=["Отдел продаж", "Подразделение"])
df["Начало месяца"] = pd.to_datetime(df['Дата']).dt.to_period('M').dt.to_timestamp()

<ipython-input-62-9a793afdf373>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Дата'] = pd.to_datetime(df['Дата'], format='%d.%m.%Y')


In [63]:
df["Должность ТА"] = df["Должность ТА"].astype(object)

In [64]:
df = df.drop(columns = ['Вес'])

In [65]:
# Продажи, руб.
sales = df.groupby(["Отдел", "Начало месяца"]).sum()
sales["Атрибут"] = "Продажи, руб."
sales


,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2024-05-01,5384559.91,"Продажи, руб."
Главмолоко,2024-05-01,28033898.69,"Продажи, руб."
ФЕРМА,2024-05-01,19521341.34,"Продажи, руб."


In [66]:
# Средняя отгрузка
avg_shimpment_df = pd.merge(sales, shimpments_qnt_df, on=["Отдел", "Начало месяца"])
avg_shimpment_df["Значение"] = avg_shimpment_df["Значение_x"] / avg_shimpment_df["Значение_y"]
avg_shimpment_df.drop(columns=["Значение_x", "Значение_y", "Атрибут_x", "Атрибут_y"], inplace=True)
avg_shimpment_df["Атрибут"] = "Средняя отгрузка"
avg_shimpment_df

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2024-05-01,2.070985e+05,Средняя отгрузка
Главмолоко,2024-05-01,1.078227e+06,Средняя отгрузка
ФЕРМА,2024-05-01,7.508208e+05,Средняя отгрузка


In [67]:
# Количество ТРТ
qnt_rtt = df.groupby(["Отдел", "Начало месяца"])["РТТ"].nunique()
qnt_rtt = qnt_rtt.to_frame(name="Значение")
qnt_rtt["Атрибут"] = "Количество ТРТ"
qnt_rtt


,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2024-05-01,63,Количество ТРТ
Главмолоко,2024-05-01,1588,Количество ТРТ
ФЕРМА,2024-05-01,1375,Количество ТРТ


In [68]:
# Количество должностей
qnt_posts = df.groupby(["Отдел", "Начало месяца"])["Должность ТА"].nunique()
qnt_posts = qnt_posts.to_frame(name="Значение")
qnt_posts["Атрибут"] = "Количество должностей"
qnt_posts

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2024-05-01,3,Количество должностей
Главмолоко,2024-05-01,20,Количество должностей
ФЕРМА,2024-05-01,13,Количество должностей


In [69]:
# Количество ТА
qnt_ta = df.groupby(["Отдел", "Начало месяца"])["ФИО ТА"].nunique()
qnt_ta = qnt_ta.to_frame(name="Значение")
qnt_ta["Атрибут"] = "Количество ТА"
qnt_ta

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2024-05-01,3,Количество ТА
Главмолоко,2024-05-01,20,Количество ТА
ФЕРМА,2024-05-01,13,Количество ТА


In [70]:
# Количество заказов
qnt_orders = df[["Отдел", "Подразделение", "Начало месяца", "РТТ", "Дата"]].drop_duplicates()
qnt_orders = qnt_orders.groupby(["Отдел", "Начало месяца"])["РТТ"].count()
qnt_orders = qnt_orders.to_frame(name="Значение")
qnt_orders["Атрибут"] = "Количество заказов"
qnt_orders

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2024-05-01,616,Количество заказов
Главмолоко,2024-05-01,8982,Количество заказов
ФЕРМА,2024-05-01,6441,Количество заказов


In [71]:
# Среднее кол-во заказов в ТРТ
avg_orders_qnt = pd.merge(qnt_orders, qnt_rtt, on=["Отдел", "Начало месяца"])
avg_orders_qnt["Значение"] = avg_orders_qnt["Значение_x"]/avg_orders_qnt["Значение_y"]
avg_orders_qnt.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avg_orders_qnt["Атрибут"] = "Среднее кол-во заказов в ТРТ"
avg_orders_qnt

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2024-05-01,9.777778,Среднее кол-во заказов в ТРТ
Главмолоко,2024-05-01,5.656171,Среднее кол-во заказов в ТРТ
ФЕРМА,2024-05-01,4.684364,Среднее кол-во заказов в ТРТ


In [72]:
# Среднее кол-во ТРТ в день на должность
avq_rtt_day_post = qnt_orders.merge(shimpments_qnt_df, on=["Отдел", "Начало месяца"]).merge(qnt_posts, on=["Отдел", "Начало месяца"])
avq_rtt_day_post["Значение"] = avq_rtt_day_post["Значение_x"]/avq_rtt_day_post["Значение_y"]/avq_rtt_day_post["Значение"]
avq_rtt_day_post.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avq_rtt_day_post["Атрибут"] = "Среднее кол-во ТРТ в день на должность"
avq_rtt_day_post

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2024-05-01,7.897436,Среднее кол-во ТРТ в день на должность
Главмолоко,2024-05-01,17.273077,Среднее кол-во ТРТ в день на должность
ФЕРМА,2024-05-01,19.056213,Среднее кол-во ТРТ в день на должность


In [73]:
# Количество ТРТ с грппировкой по ЗЗ
qnt_rtt_by_orders = df[["Отдел", "Подразделение", "Начало месяца", "РТТ", "Дата"]].drop_duplicates()
qnt_rtt_by_orders = qnt_rtt_by_orders.groupby(["Отдел", "Начало месяца", "РТТ"])["Дата"].count()
qnt_rtt_by_orders = qnt_rtt_by_orders.to_frame(name="Значение")
qnt_rtt_by_orders["Атрибут"] = "Количество ТРТ с грппировкой по ЗЗ"
qnt_rtt_by_orders["Атрибут1"] = np.where(qnt_rtt_by_orders["Значение"] < 3, "1-2 разовых заказов за месяц",
                                  np.where(qnt_rtt_by_orders["Значение"] < 5, "3-4 разовых заказов за месяц",
                                           np.where(qnt_rtt_by_orders["Значение"] < 8, "5-7 разовых заказов за месяц", 
                                                    np.where(qnt_rtt_by_orders["Значение"] >= 8, "8 и выше заказов за месяц", 0)
                                                    )
                                            )
                                  )
qnt_rtt_by_orders = qnt_rtt_by_orders.groupby(["Отдел", "Начало месяца", "Атрибут", "Атрибут1"]).count()
qnt_rtt_by_orders = qnt_rtt_by_orders.reset_index(level=["Атрибут", "Атрибут1"])
qnt_rtt_by_orders

Атрибут  \
Отдел      Начало месяца                                       
VIP        2024-05-01     Количество ТРТ с грппировкой по ЗЗ   
           2024-05-01     Количество ТРТ с грппировкой по ЗЗ   
           2024-05-01     Количество ТРТ с грппировкой по ЗЗ   
           2024-05-01     Количество ТРТ с грппировкой по ЗЗ   
Главмолоко 2024-05-01     Количество ТРТ с грппировкой по ЗЗ   
           2024-05-01     Количество ТРТ с грппировкой по ЗЗ   
           2024-05-01     Количество ТРТ с грппировкой по ЗЗ   
           2024-05-01     Количество ТРТ с грппировкой по ЗЗ   
ФЕРМА      2024-05-01     Количество ТРТ с грппировкой по ЗЗ   
           2024-05-01     Количество ТРТ с грппировкой по ЗЗ   
           2024-05-01     Количество ТРТ с грппировкой по ЗЗ   
           2024-05-01     Количество ТРТ с грппировкой по ЗЗ   

                                              Атрибут1  Значение  
Отдел      Начало месяца                                          
VIP        2024-05-01     1-2 разовых заказов за месяц         9  
           2024-05-01     3-4 разовых заказов за месяц         5  
           2024-05-01     5-7 разовых заказов за месяц         9  
           2024-05-01        8 и выше заказов за месяц        40  
Главмолоко 2024-05-01     1-2 разовых заказов за месяц       307  
           2024-05-01     3-4 разовых заказов за месяц       446  
           2024-05-01     5-7 разовых заказов за месяц       456  
           2024-05-01        8 и выше заказов за месяц       379  
ФЕРМА      2024-05-01     1-2 разовых заказов за месяц       414  
           2024-05-01     3-4 разовых заказов за месяц       418  
           2024-05-01     5-7 разовых заказов за месяц       321  
           2024-05-01        8 и выше заказов за месяц       222

In [74]:
# Средний заказ, руб.
avg_order = sales.merge(qnt_orders, on=["Отдел", "Начало месяца"])
avg_order["Значение"] = avg_order["Значение_x"]/avg_order["Значение_y"]
avg_order.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avg_order["Атрибут"] = "Средний заказ, руб."
avg_order

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2024-05-01,8741.168685,"Средний заказ, руб."
Главмолоко,2024-05-01,3121.119872,"Средний заказ, руб."
ФЕРМА,2024-05-01,3030.793563,"Средний заказ, руб."


In [75]:
# Средняя продажа в ТРТ, руб
avg_rtt_sale = sales.merge(qnt_rtt, on=["Отдел", "Начало месяца"])
avg_rtt_sale["Значение"] = avg_rtt_sale["Значение_x"]/avg_rtt_sale["Значение_y"]
avg_rtt_sale.drop(columns=["Значение_x", "Атрибут_x", "Значение_y", "Атрибут_y"], inplace=True)
avg_rtt_sale["Атрибут"] = "Средняя продажа в ТРТ, руб"
avg_rtt_sale

,,Значение,Атрибут
Отдел,Начало месяца,,
VIP,2024-05-01,85469.204921,"Средняя продажа в ТРТ, руб"
Главмолоко,2024-05-01,17653.588596,"Средняя продажа в ТРТ, руб"
ФЕРМА,2024-05-01,14197.339156,"Средняя продажа в ТРТ, руб"


In [83]:
src_ttzz_squ.head(1)

,Менеджер.Подразделение,Менеджер.СВ_Команда (Пользователи),Товарная категория,Марка (бренд),Тип товара производителя (Общие),Артикул,Номенклатура,Выручка+руч.скидка,"Вес (нетто), кг"
0,Беларусская Ферма ОП,NaN,Десерты Молочно-Творожные,Живая Марка ТП,Живая Крем сметанный,29536.0,Десерт Сметаны 13% Клубника Живая Марка ТП Вед...,128.0,0.45


In [87]:
# Активный ассортимент
df2 = src_ttzz_squ.dropna()

# Преобразование object в date
df2['Начало месяца'] = pd.to_datetime(month, format='%d.%m.%Y')

# Список полей для переименования
columns = ['Отдел продаж', 'Подразделение',	'Тип товара дистрибьютора',	'Торговая марка',	'Тип товара производителя',	'Код ТМЦ',	'ТМЦ', 'Значение', 'Вес', 'Начало месяца']
df2.columns = columns
df2.drop(columns='Вес')

df2 = pd.merge(df2, departments_df, on=["Отдел продаж", "Подразделение"])

active_assort = df2.groupby(["Отдел", "Начало месяца", "Торговая марка"])["ТМЦ"].nunique()
active_assort = active_assort.to_frame("Значение")
active_assort["Атрибут"] = "Активный ассортимент"
active_assort = active_assort.rename_axis(index={"Торговая марка" : "Атрибут1"})
active_assort = active_assort.reset_index(level= "Атрибут1")
active_assort = active_assort[["Значение", "Атрибут", "Атрибут1"]]
active_assort

<ipython-input-87-aad878bad160>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Начало месяца'] = pd.to_datetime(month, format='%d.%m.%Y')


Значение               Атрибут              Атрибут1
Отдел      Начало месяца                                                      
VIP        2024-05-01           17  Активный ассортимент             Без ТМ ТП
           2024-05-01           26  Активный ассортимент             Белавушка
           2024-05-01            3  Активный ассортимент  Белорусский Стандарт
           2024-05-01           23  Активный ассортимент            Главмолоко
           2024-05-01            4  Активный ассортимент           Доброе утро
           2024-05-01           13  Активный ассортимент        Живая Марка ТП
           2024-05-01           32  Активный ассортимент              Живая ТП
           2024-05-01            1  Активный ассортимент        Золото Полесья
           2024-05-01            4  Активный ассортимент                 Ретро
           2024-05-01           21  Активный ассортимент                 ФЕРМА
Главмолоко 2024-05-01           18  Активный ассортимент             Без ТМ ТП
           2024-05-01            4  Активный ассортимент             Белавушка
           2024-05-01            5  Активный ассортимент  Белорусский Стандарт
           2024-05-01           24  Активный ассортимент            Главмолоко
           2024-05-01            4  Активный ассортимент           Доброе утро
           2024-05-01           13  Активный ассортимент        Живая Марка ТП
           2024-05-01           36  Активный ассортимент              Живая ТП
           2024-05-01            1  Активный ассортимент        Золото Полесья
ФЕРМА      2024-05-01           17  Активный ассортимент             Без ТМ ТП
           2024-05-01           34  Активный ассортимент             Белавушка
           2024-05-01            5  Активный ассортимент  Белорусский Стандарт
           2024-05-01            3  Активный ассортимент            Главмолоко
           2024-05-01            1  Активный ассортимент           Доброе утро
           2024-05-01            4  Активный ассортимент        Живая Марка ТП
           2024-05-01            6  Активный ассортимент              Живая ТП
           2024-05-01            1  Активный ассортимент        Золото Полесья
           2024-05-01            6  Активный ассортимент                 Ретро
           2024-05-01           34  Активный ассортимент                 ФЕРМА

In [88]:
ftt_df = pd.concat([active_assort
                    , avg_rtt_sale
                    , avg_order
                    , qnt_rtt_by_orders
                    , avq_rtt_day_post
                    , avg_orders_qnt
                    , qnt_orders
                    , qnt_ta
                    , qnt_posts
                    , qnt_rtt
                    , avg_shimpment_df
                    , sales
                    , shimpments_qnt_df
                    ])

# Filter by month
view(ftt_df.loc[[("VIP", pd.to_datetime(month, format='%d.%m.%Y')) 
            , ("Главмолоко", pd.to_datetime(month, format='%d.%m.%Y'))
            , ("ФЕРМА", pd.to_datetime(month, format='%d.%m.%Y'))
            ]]
)


In [19]:
t.time() - time_stamp 

6.967608451843262